In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import re
from bson import ObjectId

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class
from AnimalShelterCRUD import AnimalShelterCRUD

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelterCRUD(username, password)

# Retrieve all records from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to avoid issues with the data table
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=250, width=251)),
    html.Center(html.B(html.H1('Alan Chumsawang SNHU CS 340 MongoDB Authentication'))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        page_size=10,
        style_table={'overflowX': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)

def update_dashboard(filter_type):
    if filter_type == 'All':
        df = pd.DataFrame.from_records(db.read({}))
    elif filter_type == 'Water':
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {"breed": {'$regex': newRegex}},
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'Mountain':
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old engilish.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': oldRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'Disaster':
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        df = pd.DataFrame.from_records(db.read({
            '$or': [
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }))
    else:
        raise Exception("Unknown filter")

    # Convert ObjectId to string
    df['_id'] = df['_id'].apply(lambda x: str(x) if isinstance(x, ObjectId) else x)

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

from dash import callback_context

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)

def update_graphs(viewData, filter_type):
    ctx = callback_context
    if not ctx.triggered:
        return [html.Div("No data available to display")]

    trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    if trigger == 'datatable-id':
        if viewData is None or len(viewData) == 0:
            return [html.Div("No data available to display")]

        dffPie = pd.DataFrame.from_dict(viewData)
        if 'breed' not in dffPie.columns:
            return [html.Div("No breed data available to display")]

        return [
            dcc.Graph(
                figure=px.pie(dffPie, names='breed', title='Preferred Animals')
            )
        ]
    elif trigger == 'filter-type':
        pass

    return [html.Div("No data available to display")]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        # Default location if no data is selected
        markerArray = (30.75, -97.48)
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
    else:
        dff = pd.DataFrame.from_dict(viewData)
        row = index[0]
        try:
            coordLat = float(dff.iloc[row]['location_lat'])
            coordLong = float(dff.iloc[row]['location_long'])
            markerArray = (coordLat, coordLong)
            toolTip = dff.iloc[row]['breed']
            popUpHeading = "Animal Name"
            popUpParagraph = dff.iloc[row]['name']
        except (ValueError, KeyError):
            markerArray = (30.75, -97.48)
            toolTip = "Austin Animal Center"
            popUpHeading = "Austin Animal Center"
            popUpParagraph = "Shelter Home Location"

    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                      ])
            ]
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:26912/
